In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.initializers import HeNormal
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
import random
from tensorflow.keras.models import load_model

def set_random_seed(seed_value=42):
    """Fix the random seed for reproducibility."""
    tf.random.set_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

# Set random seed
set_random_seed()

# Read data
data_path = 'E:/NC/FTD-Data.csv'
data = pd.read_csv(data_path, encoding='gbk')

# Extract features and targets
X = data.iloc[:, :18].values.astype(np.float32)
y = data.iloc[:, -2:].values.astype(np.float32)
concrete_types = data['CT'].values  # Used for stratified sampling

# Print counts for each concrete type
concrete_type_counts = data['CT'].value_counts()
print("Concrete type counts:\n", concrete_type_counts)

# Specify output directory and create if it does not exist
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/DNN_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initial dataset split (stratified)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, concrete_types):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    concrete_types_train, concrete_types_test = concrete_types[train_index], concrete_types[test_index]

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def build_model(input_dim, output_dim, learning_rate=0.001):
    """Build and compile a neural network model."""
    model = Sequential([
        Dense(128, input_dim=input_dim, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.2),
        Dense(32, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.2),
        Dense(output_dim)  # Output layer usually has no activation in regression
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mae')
    return model

# Dictionary to store evaluation metrics for the two targets
metrics_dict = {
    'MAE': {0: [], 1: []},
    'MSE': {0: [], 1: []},
    'RMSE': {0: [], 1: []},
    'R2': {0: [], 1: []}
}

# Dictionaries to store MAE per concrete type (optional, for further analysis)
concrete_type_mae = {ctype: [] for ctype in np.unique(concrete_types_train)}
concrete_type_counts_train = {ctype: [] for ctype in np.unique(concrete_types_train)}
concrete_type_counts_val = {ctype: [] for ctype in np.unique(concrete_types_train)}

# Lists to store fold indices (if needed later)
train_indices_list = []
val_indices_list = []

# Setup stratified K-fold cross-validation (5 folds)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
loss_history = []  # To store training history for each fold
fold_loss = []     # To store final train and validation loss for each fold

# Cross-validation training
for fold, (train_index, val_index) in enumerate(skf.split(X_train, concrete_types_train)):
    train_indices_list.append(train_index)
    val_indices_list.append(val_index)
    
    X_train_fold = X_train[train_index]
    X_val_fold = X_train[val_index]
    y_train_fold = y_train[train_index]
    y_val_fold = y_train[val_index]
    concrete_types_train_fold = concrete_types_train[train_index]
    concrete_types_val_fold = concrete_types_train[val_index]

    # Build and train the model
    model = build_model(X_train_fold.shape[1], y_train_fold.shape[1])
    history = model.fit(X_train_fold, y_train_fold, epochs=5, batch_size=64, verbose=1, 
                        validation_data=(X_val_fold, y_val_fold))
    
    # Predict on validation set
    y_pred = model.predict(X_val_fold)
    
    # Save training history for later plotting and saving CSV
    loss_history.append(history.history)
    fold_loss.append({'loss': history.history['loss'], 'val_loss': history.history['val_loss']})

    # Compute MAE for each concrete type if available in validation set
    for ctype in np.unique(concrete_types_train):
        ctype_mask = (concrete_types_val_fold == ctype)
        if np.any(ctype_mask):
            mae_value = mean_absolute_error(y_val_fold[ctype_mask], y_pred[ctype_mask])
            concrete_type_mae[ctype].append(mae_value)
            concrete_type_counts_train[ctype].append(np.sum(concrete_types_train_fold == ctype))
            concrete_type_counts_val[ctype].append(np.sum(ctype_mask))

    # Compute evaluation metrics per target
    for target in range(y_train_fold.shape[1]):
        mae_value = mean_absolute_error(y_val_fold[:, target], y_pred[:, target])
        mse_value = mean_squared_error(y_val_fold[:, target], y_pred[:, target])
        rmse_value = np.sqrt(mse_value)
        r2_value = r2_score(y_val_fold[:, target], y_pred[:, target])
        
        metrics_dict['MAE'][target].append(mae_value)
        metrics_dict['MSE'][target].append(mse_value)
        metrics_dict['RMSE'][target].append(rmse_value)
        metrics_dict['R2'][target].append(r2_value)

    # Save the training history for this fold to a CSV file
    history_df = pd.DataFrame(history.history)
    history_csv_path = os.path.join(output_dir, f'fold_{fold+1}_history.csv')
    history_df.to_csv(history_csv_path, index=False)
    print(f"Fold {fold+1} history saved to: {history_csv_path}")

    # Save the model for the current fold
    model_save_path = os.path.join(output_dir, f'model_fold_{fold+1}.h5')
    model.save(model_save_path)
    print(f"Fold {fold+1} model saved to: {model_save_path}")

# Plot training and validation loss curves for each fold
for i, history in enumerate(loss_history):
    plt.figure(figsize=(8, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves for Fold {i+1}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plot_save_path = os.path.join(output_dir, f'Loss_Curve_Fold_{i+1}.svg')
    plt.savefig(plot_save_path)
    plt.show()
    print(f"Fold {i+1} loss curve saved to: {plot_save_path}")

# Print final loss values for each fold
for fold, loss in enumerate(fold_loss):
    print(f"Fold {fold+1} - Final Training Loss: {loss['loss'][-1]:.4f}, Final Validation Loss: {loss['val_loss'][-1]:.4f}")

# Print evaluation metrics on validation sets
for metric in metrics_dict:
    for target in metrics_dict[metric]:
        mean_value = np.mean(metrics_dict[metric][target])
        print(f"{metric} for target {target}: {metrics_dict[metric][target]} (Mean: {mean_value:.4f})")

# ---------------------------
# Test set evaluation

# Dictionary to store test metrics for each fold
test_metrics = {
    'MAE': {'target1': [], 'target2': []},
    'MSE': {'target1': [], 'target2': []},
    'RMSE': {'target1': [], 'target2': []},
    'R2': {'target1': [], 'target2': []}
}

# Load each fold's model, predict on the test set, and compute metrics
for fold in range(1, skf.n_splits + 1):
    model_path = os.path.join(output_dir, f'model_fold_{fold}.h5')
    model = load_model(model_path)
    predictions = model.predict(X_test)
    
    for target in range(2):
        mae_value = mean_absolute_error(y_test[:, target], predictions[:, target])
        mse_value = mean_squared_error(y_test[:, target], predictions[:, target])
        rmse_value = np.sqrt(mse_value)
        r2_value = r2_score(y_test[:, target], predictions[:, target])
        
        test_metrics['MAE'][f'target{target+1}'].append(mae_value)
        test_metrics['MSE'][f'target{target+1}'].append(mse_value)
        test_metrics['RMSE'][f'target{target+1}'].append(rmse_value)
        test_metrics['R2'][f'target{target+1}'].append(r2_value)
        
        print(f"Fold {fold} - Target {target+1} Test Metrics:")
        print(f"  MAE: {mae_value:.4f}, MSE: {mse_value:.4f}, RMSE: {rmse_value:.4f}, R2: {r2_value:.4f}")

# Compute average test metrics across folds
average_test_metrics = {}
for metric, target_vals in test_metrics.items():
    avg_values = {}
    for target, values in target_vals.items():
        avg_values[target] = np.mean(values)
    average_test_metrics[metric] = avg_values

print("\nAverage Test Metrics:")
for metric, target_vals in average_test_metrics.items():
    for target, value in target_vals.items():
        print(f"  {metric} for {target}: {value:.4f}")

# Save test metrics (each fold and average) to a CSV file
rows = []
# Add metrics for each fold
for fold in range(1, skf.n_splits + 1):
    row = {'Fold': f'Fold {fold}'}
    for target in ['target1', 'target2']:
        row[f'MAE_{target}'] = test_metrics['MAE'][target][fold-1]
        row[f'MSE_{target}'] = test_metrics['MSE'][target][fold-1]
        row[f'RMSE_{target}'] = test_metrics['RMSE'][target][fold-1]
        row[f'R2_{target}'] = test_metrics['R2'][target][fold-1]
    rows.append(row)

# Append a row with the average metrics
avg_row = {'Fold': 'Average'}
for target in ['target1', 'target2']:
    avg_row[f'MAE_{target}'] = average_test_metrics['MAE'][target]
    avg_row[f'MSE_{target}'] = average_test_metrics['MSE'][target]
    avg_row[f'RMSE_{target}'] = average_test_metrics['RMSE'][target]
    avg_row[f'R2_{target}'] = average_test_metrics['R2'][target]
rows.append(avg_row)

test_metrics_df = pd.DataFrame(rows)
test_metrics_csv_path = os.path.join(output_dir, 'Test_Metrics.csv')
test_metrics_df.to_csv(test_metrics_csv_path, index=False)
print(f"\nTest metrics saved to: {test_metrics_csv_path}")
